In [73]:
import pandas as pd

In [158]:
# reading data in csv file using pandasb
df = pd.read_csv('filtered_tweets.csv')

In [61]:
df.head()

,target,id,date,flag,user,text
0,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
1,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
2,0,1467813579,Mon Apr 06 22:20:31 PDT 2009,NO_QUERY,starkissed,@LettyA ahh ive always wanted to see rent lov...
3,0,1467818603,Mon Apr 06 22:21:49 PDT 2009,NO_QUERY,kennypham,"Sad, sad, sad. I don't know why but I hate thi..."
4,0,1467819650,Mon Apr 06 22:22:05 PDT 2009,NO_QUERY,antzpantz,@Viennah Yay! I'm happy for you with your job!...


In [178]:
# Remove mentions, emails and links
def removeAt(tweet):
    words = tweet.split(' ')
    return ' '.join(filter(lambda x: x and '@' not in x, words))

def removeLinks(tweet):
    words = tweet.split(' ')
    return ' '.join(filter(lambda x: not ('http://' in x.lower() or 'https://' in x.lower()), words))

In [179]:
# Apply formulas to clean data
df.text = df.text.apply(lambda x: removeAt(removeLinks(x.strip())))

In [188]:
# Array of users that we removed (due to duplicates)
duplicated = df.duplicated(subset='text', keep=False)
pd.unique(df['user'])

array(['12gaBrowningGal', '15Stepz', '16_MileyCyrus', ..., 'zubinsaxena',
       'zuppalizzle', 'zzzValzzz'], dtype=object)

In [185]:
# Cleaned dataframe 
df.drop_duplicates(subset='text', keep=False, inplace=True)

# sort dataframe by user 
df.sort_values(by='user', inplace=True)
df.head()

,target,id,date,flag,user,text
14524,0,1835705341,Mon May 18 06:34:45 PDT 2009,NO_QUERY,12gaBrowningGal,Blind. Definitely. They aren't the smartest bi...
68041,0,2211076285,Wed Jun 17 12:24:07 PDT 2009,NO_QUERY,12gaBrowningGal,"I'm killing me too! I'm hungry now- Pie, Outdo..."
85046,0,2299986642,Tue Jun 23 13:35:33 PDT 2009,NO_QUERY,12gaBrowningGal,My skeet game bites! I am not joking. I will h...
203184,4,2071364168,Sun Jun 07 19:16:31 PDT 2009,NO_QUERY,12gaBrowningGal,That's just because she trying to figure out w...
207475,4,2178187758,Mon Jun 15 07:19:40 PDT 2009,NO_QUERY,12gaBrowningGal,I am really not skilled enough to shoot trap. ...


In [228]:
# Create a dictionary consolidating users and tweets to send to LIWC 
user_list = df['user'].tolist()
tweets_list = df['text'].tolist()

user_tweet_dic = {}

for i in range(len(user_list)):
    user = user_list[i]
    tweet = tweets_list[i]
    
    if user not in user_tweet_dic:
        user_tweet_dic[user] = []
        user_tweet_dic[user].append(tweet)
    else:
        user_tweet_dic[user].append(tweet)
        
new_user_list = []
new_combined_tweets_list = []

for user, tweets in user_tweet_dic.items():
    combined_tweets = ' '.join(tweets)
    if len(combined_tweets.split(' ')) >= 100:
        new_user_list.append(user)
        new_combined_tweets_list.append(combined_tweets)
    
# Create a dataframe based on dictionary and exporting as a csv file
consolidated_tweets_df = pd.DataFrame({'user': new_user_list, 'tweets': new_combined_tweets_list})
consolidated_tweets_df.to_csv('consolidated_tweets_df.csv')

<div class='alert alert-info'>
    Imagine the group taking that csv file and putting it through the LIWC software to get the output! :D
    <br>
    The output will be saved in a file called LIWC2015 Results (consolidated_tweets_df.csv).csv that will be imported later
</div>

In [230]:
# Creating a dataframe for the first 500 users 
first_500_users = consolidated_tweets_df['user'].unique()[:500]
first_500_users_df = df[df['user'].isin(first_500_users)]

# Creating a dictionary of user and their tweets 
user_tweet_dictionary = {}

# Create a format that follows IBM Watson's format and saving it to the user_tweet_dictionary
for index, row in first_500_users_df.iterrows():
    user = row['user']
    text = row['text']
    
    tweet = {}
    
    tweet['content'] = text
    tweet['contenttype'] = 'text/plain'
    tweet['language'] = 'en'
    
    if user not in user_tweet_dictionary:
        user_tweet_dictionary[user] = {'contentItems': []}
        user_tweet_dictionary[user]['contentItems'].append(tweet)
    else:
        user_tweet_dictionary[user]['contentItems'].append(tweet)

<div class='alert alert-info'>
    The bottom few cells are commented out because we are using API calls to get the output, and after a certain number of calls, we will have to pay money!
    <br>
    Instead, we have saved the file called big5_df.csv that will also be imported later
</div>

In [231]:
# import json
# from ibm_watson import PersonalityInsightsV3
# from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# #API KEY provided on the service page 
# KEY = 'STximBrq6oOabT5yZteafZcbt3m6Xh6SUSVuWr3RW3V6'

# # Authentication via IBM's IAM (Identity and Access Management)
# authenticator = IAMAuthenticator(KEY)

# # Creating a service instance
# service = PersonalityInsightsV3(
#     version='2017-10-13',
#     authenticator=authenticator)

# # Setting service endpoint 
# service.set_service_url('https://gateway.watsonplatform.net/personality-insights/api')

In [232]:
# # Creating a dictionary to store results from IBM Watson
# results = {'User': [],
#           'Openness': [],
#           'Conscientiousness': [],
#           'Extraversion': [],
#           'Agreeableness': [],
#           'Emotional range': []}

# # creates profiles of users in user_tweet_dictionary and saves them to results
# for user, tweets in user_tweet_dictionary.items():
#     profile = service.profile(tweets, 'application/json', raw_scores=True, consumption_preferences=True).get_result()
    
#     results['User'].append(user)
#     results['Openness'].append(profile['personality'][0]['raw_score'])
#     results['Conscientiousness'].append(profile['personality'][1]['raw_score'])
#     results['Extraversion'].append(profile['personality'][2]['raw_score'])
#     results['Agreeableness'].append(profile['personality'][3]['raw_score'])
#     results['Emotional range'].append(profile['personality'][4]['raw_score'])

In [249]:
# # Create a dataframe from results dictionary and save csv to use
# big5_df = pd.DataFrame(results)
# big5_df.to_csv('big5_df.csv')

In [254]:
# Importing both the outputs of LIWC and IBM Watson (Big 5)
LIWC_df = pd.read_csv('LIWC2015 Results (consolidated_tweets_df.csv).csv')
LIWC_df.drop(0, inplace=True)
LIWC_df.drop(columns=['A', 'C'], inplace=True)
LIWC_df.rename(columns={'B': 'User'}, inplace=True)
big5_df = pd.read_csv('big5_df.csv').drop(columns=['Unnamed: 0'])

# Merge dataframes together
updated_df = pd.merge(big5_df, LIWC_df)
updated_df.head()

,User,Openness,Conscientiousness,Extraversion,Agreeableness,Emotional range,WC,Analytic,Clout,Authentic,...,Comma,Colon,SemiC,QMark,Exclam,Dash,Quote,Apostro,Parenth,OtherP
0,12gaBrowningGal,0.764999,0.656144,0.548956,0.788804,0.439579,388,27.36,34.94,68.96,...,2.58,0.00,0.00,0.52,12.37,0.52,0.0,3.61,0.00,0.00
1,15Stepz,0.748124,0.690716,0.564859,0.823930,0.410770,254,26.07,70.94,84.80,...,9.45,0.39,0.00,1.18,0.39,1.57,0.0,4.33,1.57,0.00
2,16_MileyCyrus,0.672814,0.637773,0.525612,0.811025,0.563704,1164,21.04,64.42,58.80,...,3.26,0.52,0.00,2.15,13.57,0.43,0.0,1.37,0.26,1.03
3,18percentgrey,0.789412,0.607883,0.528168,0.772518,0.461528,931,47.25,53.87,58.83,...,4.83,0.64,0.75,1.61,2.26,1.93,0.0,3.22,1.29,2.26
4,19fischi75,0.706904,0.627109,0.447471,0.803232,0.518279,1707,48.43,48.83,63.95,...,0.70,0.23,0.00,3.10,1.64,3.87,0.0,0.00,2.58,3.81
